## data cleaning notebook

Goal of data cleaning (see below for further details):

1) Account for bike reshuffling between stations 

2) Generate bike availability at each station so that a predictive model can be made

We can measure the amount of bikes at a given station by the count of bike IDs at a station at that point in time. Note that since availability at a given station depends on the starting number of bikes at that station, we need to do a similar analysis for prior data in order to get a starting point of how many bikes initially began at a given station.

In [ ]:
# packages
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
files = ['2019' +'0' + str(i) + '-capitalbikeshare-tripdata.csv' if len(str(i)) == 1 else '2019' + str(i) + '-capitalbikeshare-tripdata.csv' for i in range(1,13)]

In [ ]:
data = pd.concat([pd.read_csv(f, parse_dates = ['Start date', 'End date']) for f in files])

In [ ]:
## cleaning col names
data.columns = ["_".join(col.split(" ")).capitalize() for col in data.columns]
data.columns

In [ ]:
data.Duration.describe().round(2)

In [ ]:
## subsetting relevant columns for analysis
data = data.loc[:, "Start_date":"Bike_number"]
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
old = pd.read_csv('201812-capitalbikeshare-tripdata.csv', parse_dates = ['Start date', 'End date'])

In [ ]:
## cleaning col names
old.columns = ["_".join(col.split(" ")).capitalize() for col in old.columns]
old.columns

In [ ]:
old = old.loc[:, "Start_date":"Bike_number"]
old.head()

In [ ]:
old.shape

In [ ]:
old.dtypes

In [ ]:
old.info()

In [ ]:
old.isnull().sum()

### reshuffling

Idea: bikes transition from one station to another without a user taking the trip (this is done by contractors who 'reshuffle' vans from one location to the next). To account for this, we need to add rows that account for this reshuffling in order to get a more accurate count of where bikes are and when they're being relocated in order to calculate availability.

logic for reshuffling of bike X:

- if A --> B and C --> D, X is reshuffled from B to C

- A => B, B => Van, Van => C, ...

In [ ]:
data.Bike_number.isnull().sum() # NAs present, dropping from analysis

In [ ]:
## variable for looping
r_data = data[~data.Bike_number.isna()].sort_values(['Bike_number', 'Start_date']).reset_index(drop = True)
r_data.head()

In [ ]:
## logic for reshuffling of bike X:
## if A --> B and C --> D, X is reshuffled from B to C
# A => B, B => Van, Van => C, ...

data_shuf_list = []
i = 0
while i < (len(r_data) - 1):
    if r_data.loc[i, "End_station_number"] != r_data.loc[i + 1, "Start_station_number"] and r_data.loc[i, "Bike_number"] == r_data.loc[i + 1, "Bike_number"]:
        # reshuffling # row 1
        data_shuf_list.append((r_data.loc[i, "Start_date"], r_data.loc[i, "End_date"],
             # reshuffling line
             r_data.loc[i, "End_station_number"], r_data.loc[i, "End_station"], # new start
             1, "Van", # new end
             r_data.loc[i, "Bike_number"]
            ))
        # row 2
        data_shuf_list.append((r_data.loc[i, "Start_date"], r_data.loc[i, "End_date"],
             # reshuffling line
             1, "Van", # new start
             r_data.loc[i + 1, "Start_station_number"],r_data.loc[i + 1, "End_station"], # new end
             r_data.loc[i, "Bike_number"]
            ))
    i += 1 # updating loop

In [ ]:
data_shuf_df = pd.DataFrame(data_shuf_list, columns = data.columns)
data_shuf_df.shape

In [ ]:
data_shuf_df.head()

In [ ]:
data_shuf_df.isna().sum()

In [ ]:
# data_shuf_df.to_csv('reshuffled_2019.csv') - writing to csv file

## applying to previous data

In [ ]:
## variable for looping
r_old = old[~old.Bike_number.isna()].sort_values(['Bike_number', 'Start_date']).reset_index(drop = True)
r_old.head()

In [ ]:
## row reshuffling - old data (similar logic to regular data)

old_shuf_list = []
i = 0
while i < (len(r_old) - 1):
    if r_old.loc[i, "End_station_number"] != r_old.loc[i + 1, "Start_station_number"] and r_old.loc[i, "Bike_number"] == r_old.loc[i + 1, "Bike_number"]:
        # reshuffling
        # row 1
        old_shuf_list.append((r_old.loc[i, "Start_date"], r_old.loc[i, "End_date"],
             # reshuffling line
             r_old.loc[i, "End_station_number"], r_old.loc[i, "End_station"], # this is the new start
             1, "Van", # this is the new end
             r_old.loc[i, "Bike_number"]
            ))
        # row 2
        old_shuf_list.append((r_old.loc[i, "Start_date"], r_old.loc[i, "End_date"],
             # reshuffling line
             1, "Van", # this is the new start
             r_old.loc[i + 1, "Start_station_number"],r_old.loc[i + 1, "End_station"], # this is the new end
             r_old.loc[i, "Bike_number"]
                ))
    i += 1 # updating loop

In [ ]:
old_shuf_df = pd.DataFrame(old_shuf_list, columns = old.columns)
old_shuf_df.shape

In [ ]:
old_shuf_df.head()

In [ ]:
old_shuf_df.isna().sum()

In [ ]:
sorted(old_shuf_df.Start_station_number.unique().tolist())

In [ ]:
sorted(data_shuf_df.Start_station_number.unique().tolist())

In [ ]:
len(set(sorted(data_shuf_df.Start_station_number.unique().tolist())).difference(set(sorted(old_shuf_df.Start_station_number.unique().tolist()))))

In [ ]:
# old_shuf_df.to_csv('reshuffled_2018.csv')

### adding reshuffled data to the data frames

Now that reshuffling is finished, we can append these rows onto the original dataframes. This will account for any reshuffling of the bikes from each station that occurred.

In [ ]:
## 2019 data
combined_data = pd.concat([data, data_shuf_df], axis = 0)
combined_data.head()

In [ ]:
combined_data.shape

In [ ]:
combined_data.isna().sum()

In [ ]:
#combined_data.to_csv("combined_2019.csv")

In [ ]:
## 2018 data
combined_old = pd.concat([old, old_shuf_df], axis = 0)
combined_old.isna().sum()

In [ ]:
combined_old.shape

In [ ]:
#combined_old.to_csv("combined_2018.csv")

# Availability

Calculating bike availability after accounting for bike reshuffling (can skip down to this code directly for analysis).

In [1]:
# packages
import pandas as pd
import datetime as dt
import numpy as np

In [4]:
combined_data = pd.read_csv("combined_2019.csv", parse_dates = ["Start_date", "End_date"], index_col = 0)
combined_data.head()

C:\Users\Owner\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Start_date,End_date,Start_station_number,Start_station,End_station_number,End_station,Bike_number
0,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141
1,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067
2,2019-01-01 00:08:46,2019-01-01 00:11:44,31104,Adams Mill & Columbia Rd NW,31323,Woodley Park Metro / Calvert St & Connecticut ...,W22654
3,2019-01-01 00:08:47,2019-01-01 00:12:35,31281,8th & O St NW,31280,11th & S St NW,W22336
4,2019-01-01 00:12:29,2019-01-01 00:34:10,31014,Lynn & 19th St North,31923,Columbia Pike & S Taylor St,70004


In [5]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4095805 entries, 0 to 697387
Data columns (total 7 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Start_date            datetime64[ns]
 1   End_date              datetime64[ns]
 2   Start_station_number  int64         
 3   Start_station         object        
 4   End_station_number    int64         
 5   End_station           object        
 6   Bike_number           object        
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 250.0+ MB


In [7]:
combined_data.shape

(4095805, 7)

In [8]:
combined_data.isnull().sum()

Start_date               0
End_date                 0
Start_station_number     0
Start_station            0
End_station_number       0
End_station              0
Bike_number             14
dtype: int64

In [9]:
combined_old = pd.read_csv("combined_2018.csv", parse_dates = ["Start_date", "End_date"], index_col = 0)
combined_old.head()

,Start_date,End_date,Start_station_number,Start_station,End_station_number,End_station,Bike_number
0,2018-12-01 00:00:44,2018-12-01 00:13:54,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21477
1,2018-12-01 00:00:55,2018-12-01 00:13:46,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W22810
2,2018-12-01 00:00:55,2018-12-01 00:13:51,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21144
3,2018-12-01 00:01:08,2018-12-01 00:13:39,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W23321
4,2018-12-01 00:02:45,2018-12-01 00:05:11,31124,14th & Irving St NW,31102,11th & Kenyon St NW,W22304


In [10]:
combined_old.dtypes

Start_date              datetime64[ns]
End_date                datetime64[ns]
Start_station_number             int64
Start_station                   object
End_station_number               int64
End_station                     object
Bike_number                     object
dtype: object

In [11]:
combined_old.head()

,Start_date,End_date,Start_station_number,Start_station,End_station_number,End_station,Bike_number
0,2018-12-01 00:00:44,2018-12-01 00:13:54,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21477
1,2018-12-01 00:00:55,2018-12-01 00:13:46,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W22810
2,2018-12-01 00:00:55,2018-12-01 00:13:51,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W21144
3,2018-12-01 00:01:08,2018-12-01 00:13:39,31203,14th & Rhode Island Ave NW,31116,California St & Florida Ave NW,W23321
4,2018-12-01 00:02:45,2018-12-01 00:05:11,31124,14th & Irving St NW,31102,11th & Kenyon St NW,W22304


In [12]:
combined_old.isna().sum()

Start_date              0
End_date                0
Start_station_number    0
Start_station           0
End_station_number      0
End_station             0
Bike_number             0
dtype: int64

In [13]:
## creating appropriate cols for analysis
print(combined_data.shape)
combined_data['Start_day'] = combined_data.Start_date.dt.date
combined_data['Start_hour'] = combined_data.Start_date.dt.hour
combined_data.shape

(4095805, 7)


(4095805, 9)

In [14]:
print(combined_old.shape)
combined_old['Start_day'] = combined_old.Start_date.dt.date
combined_old['Start_hour'] = combined_old.Start_date.dt.hour
combined_old.shape

(195107, 7)


(195107, 9)

# availability calculation

## step 1: get arrivals and departures to a station

Idea: since we want number of bikes at a station by hour, we group the data together with these and count the number of bikes that are present at station using bike_number.

Arrivals: the bike ends at a station at the start of the hour

Departures: the bike starts at a station at the start of the hour

In [15]:
comb_a = combined_data.groupby(['Start_day', 'Start_hour', 'End_station_number'])['Bike_number'].count().reset_index()\
.rename(columns = {"End_station_number":"Station", "Bike_number":"Arrivals"})
comb_a.head()

,Start_day,Start_hour,Station,Arrivals
0,2019-01-01,0,1,9
1,2019-01-01,0,31030,1
2,2019-01-01,0,31032,1
3,2019-01-01,0,31041,1
4,2019-01-01,0,31048,1


In [16]:
comb_a.shape

(1353200, 4)

In [17]:
comb_d = combined_data.groupby(['Start_day', 'Start_hour', 'Start_station_number'])['Bike_number'].count().reset_index()\
.rename(columns = {"Start_station_number":"Station", "Bike_number":"Departures"})
comb_d.head()

,Start_day,Start_hour,Station,Departures
0,2019-01-01,0,1,9
1,2019-01-01,0,31014,1
2,2019-01-01,0,31022,1
3,2019-01-01,0,31041,1
4,2019-01-01,0,31042,1


In [18]:
comb_d.shape

(1348500, 4)

## merging tables

We then merge the tables together using the date, time, and station number to combine the columns. Note that we use an outer join and fill in the missing values with 0, since this implies that there were only arrivals/departures to a station at a given point in time.

In [19]:
comb_a_d = comb_a.merge(comb_d, how = "outer", on = ["Start_day", "Start_hour", "Station"]).fillna(0)
comb_a_d.shape

(1785501, 5)

In [20]:
comb_a_d.head()

,Start_day,Start_hour,Station,Arrivals,Departures
0,2019-01-01,0,1,9.0,9.0
1,2019-01-01,0,31030,1.0,0.0
2,2019-01-01,0,31032,1.0,0.0
3,2019-01-01,0,31041,1.0,1.0
4,2019-01-01,0,31048,1.0,0.0


In [21]:
comb_a_d[comb_a_d.Station == 31030].head() # sanity check that combining worked

,Start_day,Start_hour,Station,Arrivals,Departures
1,2019-01-01,0,31030,1.0,0.0
1576,2019-01-01,16,31030,2.0,2.0
1778,2019-01-01,17,31030,3.0,0.0
1944,2019-01-01,18,31030,1.0,0.0
2389,2019-01-01,23,31030,1.0,0.0


### starting point calculation 

identical to analysis for previous data to get in a similar format

In [22]:
# arrivals
old_a = combined_old.groupby(['Start_day', 'Start_hour', 'End_station_number'])['Bike_number'].count().reset_index()\
.rename(columns = {"End_station_number":"Station", "Bike_number":"Arrivals"})
old_a.head()

,Start_day,Start_hour,Station,Arrivals
0,2018-12-01,0,1,11
1,2018-12-01,0,31009,1
2,2018-12-01,0,31020,1
3,2018-12-01,0,31030,1
4,2018-12-01,0,31052,1


In [23]:
old_a.shape

(81433, 4)

In [24]:
# departures
old_d = combined_old.groupby(['Start_day', 'Start_hour', 'Start_station_number'])['Bike_number'].count().reset_index()\
.rename(columns = {"Start_station_number":"Station", "Bike_number":"Departures"})
old_d.head()

,Start_day,Start_hour,Station,Departures
0,2018-12-01,0,1,11
1,2018-12-01,0,31007,2
2,2018-12-01,0,31009,1
3,2018-12-01,0,31034,1
4,2018-12-01,0,31037,1


In [25]:
old_d.shape

(81357, 4)

In [26]:
old_a_d = old_a.merge(old_d, how = "outer", on = ["Start_day", "Start_hour", "Station"]).fillna(0)
old_a_d.head()

,Start_day,Start_hour,Station,Arrivals,Departures
0,2018-12-01,0,1,11.0,11.0
1,2018-12-01,0,31009,1.0,1.0
2,2018-12-01,0,31020,1.0,0.0
3,2018-12-01,0,31030,1.0,0.0
4,2018-12-01,0,31052,1.0,0.0


In [27]:
old_a_d.shape

(112263, 5)

#### actual calculation for starting point

look at bike at each station, try to identify bikes that arrived and departed at that station ==> count bikes that have departure before arrival

In [29]:
old_a_d.rename({"Start_day":"Start_date"}, axis = 1, inplace = True)

#### arrivals departures

Find the last index value of each bike grouped by station: this gives a sense of where the bikes where at their final trip for each station.

In [30]:
# departure starting point for each bike
dep_start = combined_old[["Bike_number",'Start_date', 'Start_station_number']]\
.loc[combined_old.groupby(['Start_station_number','Bike_number'])['Start_date'].idxmax()]\
.reset_index(drop = True).rename(columns = {'Start_date':'Max_depart', 'Start_station_number':'Station_id'})
dep_start.head()

,Bike_number,Max_depart,Station_id
0,W22810,2018-12-01 00:00:55,31203
1,23750,2018-12-01 01:33:45,1
2,W23924,2018-12-01 00:09:06,31636
3,23751,2018-12-11 07:48:44,1
4,W20167,2018-12-01 00:11:16,31400


In [31]:
# arrival starting point for each bike
arrive_start = combined_old[["Bike_number",'Start_date', 'End_station_number']]\
.loc[combined_old.groupby(['End_station_number','Bike_number'])['Start_date'].idxmax()]\
.reset_index(drop = True).rename(columns = {'Start_date':'Max_arrive', 'End_station_number':'Station_id'})
arrive_start.head()

,Bike_number,Max_arrive,Station_id
0,W21477,2018-12-01 00:00:44,31116
1,23750,2018-12-01 01:33:45,1
2,W21353,2018-12-01 00:08:24,31223
3,23751,2018-12-11 07:48:44,1
4,W00539,2018-12-01 00:10:55,31286


### checking that bikes are present in both data sets

For ease of comparison, include only bikes in both data sets when tracking starting point over time. Assume that bikes in the old data set and not the new one were removed or broken.

In [32]:
# 19 bikes are present in the old data and not the new data, so we assume that they are removed
print(combined_data.Bike_number.nunique())
print(combined_old.Bike_number.nunique())
len(set(combined_old.Bike_number.unique()).intersection(set(combined_data.Bike_number.unique())))

6052
3982


3963

In [33]:
# filter on bikes only present in 2019 data
arrive_start_good = arrive_start.loc[arrive_start['Bike_number'].isin(combined_data['Bike_number'].unique())]
dep_start_good = dep_start.loc[dep_start['Bike_number'].isin(combined_data['Bike_number'].unique())]

In [34]:
arrive_start_good.head()

,Bike_number,Max_arrive,Station_id
0,W21477,2018-12-01 00:00:44,31116
2,W21353,2018-12-01 00:08:24,31223
4,W00539,2018-12-01 00:10:55,31286
6,W22558,2018-12-01 00:12:49,31110
7,51020,2018-12-27 07:24:03,1


In [35]:
dep_start_good.head()

,Bike_number,Max_depart,Station_id
0,W22810,2018-12-01 00:00:55,31203
2,W23924,2018-12-01 00:09:06,31636
4,W20167,2018-12-01 00:11:16,31400
6,W23850,2018-12-01 00:12:50,31104
7,51020,2018-12-27 07:24:03,1


In [36]:
# Merge the arrivals and departure together and identify location of bikes at each station
last_bike = arrive_start_good.merge(dep_start_good, how = 'outer', on = ['Bike_number', "Station_id"])
last_bike.head()

,Bike_number,Max_arrive,Station_id,Max_depart
0,W21477,2018-12-01 00:00:44,31116,2018-12-01 13:42:31
1,W21477,2018-12-01 10:06:34,31116,2018-12-01 13:42:31
2,W21477,2018-12-01 13:42:31,31116,2018-12-01 13:42:31
3,W21477,2018-12-01 13:42:31,31116,2018-12-01 13:42:31
4,W21353,2018-12-01 00:08:24,31223,2018-12-01 00:08:24


In [37]:
last_bike.shape

(311943, 4)

In [38]:
last_bike.isnull().sum()

Bike_number       0
Max_arrive     4157
Station_id        0
Max_depart     2867
dtype: int64

### flagging bikes that are present at a station for end of the old data

Idea: if the arrival time is after the departure time, then the bike is at a station; if the most recent departure is NA (i.e. there hasn't been a trip made), then the bike is at a station

We assign 0s or 1s to make it easy to count the number of bikes at each station as a starting point.

In [39]:
last_bike['check'] = np.where(((last_bike.Max_arrive > last_bike.Max_depart) | (last_bike.Max_depart.isna())) , 1, 0)

In [40]:
last_bike_df = last_bike.groupby("Station_id")['check'].sum().reset_index()

In [41]:
## adding columns to data frame for time consistency (getting last available observation)
# we can then use this dataframe to match up with combined data that can be appended
last_bike_df['Start_day'] = dt.date(day = 31, month = 12, year = 2018)
last_bike_df['Start_hour'] = 23
last_bike_df['Departures'] = 0
last_bike_df.rename(columns = {'check':'Arrivals'}, inplace = True)
last_bike_df.rename(columns = {"Station_id":"Station"}, inplace = True)
last_bike_df = last_bike_df.reindex(columns = comb_a_d.columns)
last_bike_df['Diff'] = last_bike_df.Arrivals.sub(last_bike_df.Departures)
last_bike_df.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff
0,2018-12-31,23,1,38283,0,38283
1,2018-12-31,23,31000,6,0,6
2,2018-12-31,23,31001,8,0,8
3,2018-12-31,23,31002,24,0,24
4,2018-12-31,23,31003,12,0,12


### calculating arrival and departure differences

In [42]:
comb_a_d['Diff'] = comb_a_d['Arrivals'].sub(comb_a_d['Departures'])
comb_a_d.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff
0,2019-01-01,0,1,9.0,9.0,0.0
1,2019-01-01,0,31030,1.0,0.0,1.0
2,2019-01-01,0,31032,1.0,0.0,1.0
3,2019-01-01,0,31041,1.0,1.0,0.0
4,2019-01-01,0,31048,1.0,0.0,1.0


In [43]:
## appending the data
full = pd.concat([last_bike_df, comb_a_d], axis = 0)
full.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff
0,2018-12-31,23,1,38283.0,0.0,38283.0
1,2018-12-31,23,31000,6.0,0.0,6.0
2,2018-12-31,23,31001,8.0,0.0,8.0
3,2018-12-31,23,31002,24.0,0.0,24.0
4,2018-12-31,23,31003,12.0,0.0,12.0


In [44]:
full.shape

(1786026, 6)

In [45]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1786026 entries, 0 to 1785500
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Start_day   object 
 1   Start_hour  int64  
 2   Station     int64  
 3   Arrivals    float64
 4   Departures  float64
 5   Diff        float64
dtypes: float64(3), int64(2), object(1)
memory usage: 95.4+ MB


In [46]:
full.isnull().sum()

Start_day     0
Start_hour    0
Station       0
Arrivals      0
Departures    0
Diff          0
dtype: int64

### Including station capacity

We now have the number of bikes present at each station, and want to understand the availability relative to capacity ratio for further analysis.

In [47]:
capacity = pd.read_csv('Capital_Bike_Share_Locations.csv',
                       usecols = ["NAME", "LATITUDE", "LONGITUDE", "CAPACITY", "REGION_NAME"]) # usecols and pass a list []
capacity.columns = ["_".join(col.split(" ")).capitalize() for col in capacity.columns]
capacity.head()

,Latitude,Longitude,Name,Capacity,Region_name
0,38.898984,-77.078317,21st St & N Pierce St,15,"Arlington, VA"
1,38.858971,-77.053230,Eads St & 15th St S,15,"Arlington, VA"
2,38.856425,-77.049232,Crystal Dr & 20th St S,17,"Arlington, VA"
3,38.861056,-77.049417,Crystal Dr & 15th St S,16,"Arlington, VA"
4,38.857866,-77.059490,Aurora Hills Cmty Ctr / 18th St & S Hayes St,12,"Arlington, VA"


In [69]:
# some stations in the data set are missing from the capacity data set (94) - these will be removed
combined_data[["Start_station_number", "Start_station"]].drop_duplicates(subset = ['Start_station_number'])\
.merge(capacity, how = 'left',left_on = "Start_station", right_on = "Name").isnull().sum()

Start_station_number     0
Start_station            0
Latitude                94
Longitude               94
Name                    94
Capacity                94
Region_name             94
dtype: int64

In [62]:
# check that van is presenet
1 in combined_data[["Start_station_number", "Start_station"]].drop_duplicates(subset = ['Start_station_number'])["Start_station_number"]

True

In [70]:
# first merge adds on station name, second adds on the other columns from capacity (location, capacity, etc.)
full_bike = full.merge(combined_data[["Start_station_number", "Start_station"]].drop_duplicates(subset = ['Start_station_number']),
          how = 'left', left_on = "Station", right_on = "Start_station_number")\
.merge(capacity, how = "left", left_on = "Start_station", right_on = "Name")
full_bike.isnull().sum()

Start_day                    0
Start_hour                   0
Station                      0
Arrivals                     0
Departures                   0
Diff                         0
Start_station_number         0
Start_station                0
Latitude                240095
Longitude               240095
Name                    240095
Capacity                240095
Region_name             240095
dtype: int64

In [71]:
1 in full_bike.Start_station_number.unique()

True

In [72]:
full_bike.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,Name,Capacity,Region_name
0,2018-12-31,23,1,38283.0,0.0,38283.0,1,Van,NaN,NaN,NaN,NaN,NaN
1,2018-12-31,23,31000,6.0,0.0,6.0,31000,Eads St & 15th St S,38.858971,-77.053230,Eads St & 15th St S,15.0,"Arlington, VA"
2,2018-12-31,23,31001,8.0,0.0,8.0,31001,18th & Eads St.,NaN,NaN,NaN,NaN,NaN
3,2018-12-31,23,31002,24.0,0.0,24.0,31002,Crystal Dr & 20th St S,38.856425,-77.049232,Crystal Dr & 20th St S,17.0,"Arlington, VA"
4,2018-12-31,23,31003,12.0,0.0,12.0,31003,Crystal Dr & 15th St S,38.861056,-77.049417,Crystal Dr & 15th St S,16.0,"Arlington, VA"


In [75]:
full_bike.columns

Index(['Start_day', 'Start_hour', 'Station', 'Arrivals', 'Departures', 'Diff',
       'Start_station_number', 'Start_station', 'Latitude', 'Longitude',
       'Name', 'Capacity', 'Region_name'],
      dtype='object')

### data validation

In [77]:
# Sort the values so that we can view the stations individually one at a time in chronological order
full_bike.sort_values(['Station','Start_day', 'Start_hour'], inplace = True)
full_bike.reset_index(drop = True, inplace = True)
full_bike.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,Name,Capacity,Region_name
0,2019-11-26,9,0,2.0,0.0,2.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC"
1,2019-11-27,9,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC"
2,2019-11-28,16,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC"
3,2019-11-30,23,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC"
4,2019-12-01,16,0,0.0,1.0,-1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC"


In [85]:
full_bike[['Station','Start_day', 'Start_hour','Diff']]\
.groupby(['Station','Start_day', 'Start_hour'])\
.sum().groupby(level=0).cumsum().reset_index()

,Station,Start_day,Start_hour,Diff
0,0,2019-11-26,9,2.0
1,0,2019-11-27,9,3.0
2,0,2019-11-28,16,4.0
3,0,2019-11-30,23,5.0
4,0,2019-12-01,16,4.0
...,...,...,...,...
1786021,32609,2019-12-26,13,-10.0
1786022,32609,2019-12-28,11,-10.0
1786023,32609,2019-12-28,13,-8.0
1786024,32609,2019-12-28,14,-10.0


In [86]:
## by station by day by hour, calculate cumulative sum of bikes present at a location
full_bike['cumu_sum1'] = full_bike[['Station','Start_day', 'Start_hour','Diff']]\
.groupby(['Station','Start_day', 'Start_hour'])\
.sum().groupby(level=0).cumsum().reset_index()['Diff']

In [87]:
## cumulative sum validation
station_num = full_bike.loc[0,'Station']
cumu_sum = 0
zero_counter = 0
cap_counter = 0
for i in range(0,len(full_bike)):
    # Check to see if the station number has changed
    if full_bike.loc[i,'Station'] != station_num:
        # If it has, reset the sum to the new station
        station_num = full_bike.loc[i,'Station']
        cumu_sum = 0
    # If the cumu sum has reached max capacity
    if cumu_sum + full_bike.loc[i, 'Diff'] >= full_bike.loc[i,'Capacity']:
        full_bike.at[i,'cumu_sum2'] = full_bike.loc[i,'Capacity']
        cumu_sum = full_bike.loc[i,'Capacity']
        cap_counter +=1
    # If the cumu sum has reached minimum capacity
    elif cumu_sum + full_bike.loc[i, 'Diff'] < 0:
        full_bike.at[i,'cumu_sum2'] = 0
        cumu_sum = 0
        zero_counter += 1
    else:
        # Add to the cumulative sum and add the data point to the table
        cumu_sum += full_bike.loc[i, 'Diff'] 
        full_bike.at[i,'cumu_sum2'] = cumu_sum

In [96]:
print("Count restricted cum sum < 0: {}".format(zero_counter))
print("Count restricted cum sum > capacity: {}".format(cap_counter))
print("Proprotion of bikes affected by restriction: {}".format((zero_counter + cap_counter)/full_bike.shape[0]))

Count restricted cum sum < 0: 14243
Count restricted cum sum > capacity: 32768
Proprotion of bikes affected by restriction: 0.026321565307559913


In [90]:
print("Count of raw sum > capacity: {}".format(full_bike[abs(full_bike['cumu_sum1']) > full_bike['Capacity']].shape[0]))
print("Count of raw sum < 0: {}".format(full_bike[abs(full_bike['cumu_sum1']) < 0].shape[0]))

Count of raw sum > capacity: 931478
Count of raw sum < 0: 0


In [99]:
full_bike.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,Name,Capacity,Region_name,cumu_sum1,cumu_sum2
0,2019-11-26,9,0,2.0,0.0,2.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",2.0,2.0
1,2019-11-27,9,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",3.0,3.0
2,2019-11-28,16,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",4.0,4.0
3,2019-11-30,23,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",5.0,5.0
4,2019-12-01,16,0,0.0,1.0,-1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",4.0,4.0


## explaining unusual behavior

https://www.wnyc.org/story/285378-nine-percent-of-all-d-c-bike-share-bikes-are-at-washington-nationals-stadium-for-the-playoff-game/

Bikeshare allows overflow capacity to a station during special events.

In [100]:
num_bad_starting_points = full_bike[(full_bike['Start_day'] == dt.date(month = 12, year = 2018, day = 31))&(full_bike['Arrivals']>full_bike['Capacity'])].shape[0]
print("Count of stations with a starting number > capacity: {}".format(num_bad_starting_points))

Count of stations with a starting number > capacity: 177


## adding more data

need to clean up variable names for the data

In [106]:
full_bike.drop(columns = ['cumu_sum1'], axis = 1, inplace = True)
full_bike.rename({'cumu_sum2':'cumu_sum'}, axis = 1, inplace = True)
full_bike['percent_full'] = full_bike['cumu_sum']/full_bike['Capacity']
full_bike['weekday_num'] = pd.to_datetime(full_bike['Start_day'], errors = 'coerce').dt.weekday
full_bike.head()

KeyError: "['cumu_sum1'] not found in axis"

In [107]:
full_bike.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,Name,Capacity,Region_name,cumu_sum,percent_full,weekday_num
0,2019-11-26,9,0,2.0,0.0,2.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",2.0,0.117647,1
1,2019-11-27,9,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",3.0,0.176471,2
2,2019-11-28,16,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",4.0,0.235294,3
3,2019-11-30,23,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",5.0,0.294118,5
4,2019-12-01,16,0,0.0,1.0,-1.0,0,22nd & H St NW,38.898925,-77.048852,22nd & H St NW,17.0,"Washington, DC",4.0,0.235294,6


In [114]:
# commands are identical
#np.sum(np.where((full_bike['weekday_num'] > 4), 1, 0) == full_bike['weekday_num'].apply(lambda x: 1 if x > 4 else 0).values) == full_bike.shape[0]
full_bike['weekend_flag'] = full_bike['weekday_num'].apply(lambda x: 1 if x > 4 else 0).values

In [117]:
## adding weather data - drop last column because redundant
weather = pd.read_csv('weather.csv', parse_dates = ["Date"]).iloc[:,0:-1]
weather.columns = [col.split(" ")[0] for col in weather.columns]
weather.head()

,Date,TAVG,TMAX,TMIN,PRCP,SNOW
0,2018-12-31,44,51,39,0.48,0.0
1,2019-01-01,56,64,44,0.00,0.0
2,2019-01-02,46,48,42,0.00,0.0
3,2019-01-03,47,51,40,0.00,0.0
4,2019-01-04,43,50,36,0.04,0.0


In [118]:
weather.dtypes

Date    datetime64[ns]
TAVG             int64
TMAX             int64
TMIN             int64
PRCP           float64
SNOW           float64
dtype: object

In [119]:
# merging weather data onto data frame
full_bike['Start_day'] = pd.to_datetime(full_bike['Start_day'])
full_bike_data = pd.merge(full_bike, weather, how = 'left', left_on = 'Start_day', right_on = "Date")
full_bike_data.head()

,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,...,cumu_sum,percent_full,weekday_num,weekend_flag,Date,TAVG,TMAX,TMIN,PRCP,SNOW
0,2019-11-26,9,0,2.0,0.0,2.0,0,22nd & H St NW,38.898925,-77.048852,...,2.0,0.117647,1,0,2019-11-26,48,62,38,0.00,0.0
1,2019-11-27,9,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,3.0,0.176471,2,0,2019-11-27,51,58,44,0.01,0.0
2,2019-11-28,16,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,4.0,0.235294,3,0,2019-11-28,50,54,43,0.00,0.0
3,2019-11-30,23,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,5.0,0.294118,5,1,2019-11-30,43,46,41,0.00,0.0
4,2019-12-01,16,0,0.0,1.0,-1.0,0,22nd & H St NW,38.898925,-77.048852,...,4.0,0.235294,6,1,2019-12-01,43,46,40,0.50,0.0


In [120]:
# adding dates onto the dataframe - federal holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start= full_bike['Start_day'].min(), end=full_bike['Start_day'].max())
holidays

DatetimeIndex(['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27',
               '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11',
               '2019-11-28', '2019-12-25'],
              dtype='datetime64[ns]', freq=None)

In [121]:
full_bike_data['Holiday'] = pd.to_datetime(full_bike_data['Start_day']).isin(holidays)

In [128]:
## cleaning up columns - only keep ones relevant to analysis
print(len(full_bike_data.columns))
full_bike_data.head()

24


,Start_day,Start_hour,Station,Arrivals,Departures,Diff,Start_station_number,Start_station,Latitude,Longitude,...,percent_full,weekday_num,weekend_flag,Date,TAVG,TMAX,TMIN,PRCP,SNOW,Holiday
0,2019-11-26,9,0,2.0,0.0,2.0,0,22nd & H St NW,38.898925,-77.048852,...,0.117647,1,0,2019-11-26,48,62,38,0.00,0.0,False
1,2019-11-27,9,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,0.176471,2,0,2019-11-27,51,58,44,0.01,0.0,False
2,2019-11-28,16,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,0.235294,3,0,2019-11-28,50,54,43,0.00,0.0,True
3,2019-11-30,23,0,1.0,0.0,1.0,0,22nd & H St NW,38.898925,-77.048852,...,0.294118,5,1,2019-11-30,43,46,41,0.00,0.0,False
4,2019-12-01,16,0,0.0,1.0,-1.0,0,22nd & H St NW,38.898925,-77.048852,...,0.235294,6,1,2019-12-01,43,46,40,0.50,0.0,False


In [130]:
keepers = ["Start_day", "Start_hour", "Arrivals", "Departures", "Diff", 'Start_station_number', "Start_station", "Latitude", "Longitude", "Capacity", "Region_name", "cumu_sum", "percent_full", "weekend_flag", "TAVG", "TMAX", "TMIN", "PRCP", "SNOW", "Holiday"]

In [131]:
len(keepers)

20

In [132]:
clean_data = full_bike_data[keepers]

In [133]:
#clean_data.to_csv("bikeshare_data.csv")